# Splits
Validate the splits file for fold/train/valid consistency.

In [1]:
import datetime
print(datetime.datetime.now())
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

2022-08-01 11:52:59.099200


In [2]:
PATHNAME='/home/jrm/Adjeroh/Glioma/August_Run/TrainValidSplits/'
FILENAME='aug_train_valid_splits.csv'
INFILE=PATHNAME+FILENAME

In [3]:
df = pd.read_csv(INFILE)
df

,Fold,Split,Case,Class
0,0,Train,TCGA-02-0004-01Z-00,0
1,0,Train,TCGA-02-0010-01Z-00,0
2,0,Train,TCGA-14-0789-01Z-00,0
3,0,Train,TCGA-02-0025-01Z-00,0
4,0,Train,TCGA-02-0033-01Z-00,0
...,...,...,...,...
630,4,Valid,TCGA-S9-A7J2-01Z-00,5
631,4,Valid,TCGA-02-0430-01Z-00,0
632,4,Valid,TCGA-28-1746-01Z-00,0
633,4,Valid,TCGA-HT-7676-01Z-00,1


In [4]:
class all_splits():
    def load_csv(self,FULLPATH):
        self.FOLDS=5
        empty = []
        self.X_train_sets=[[] for i in range(self.FOLDS)]
        self.y_train_sets=[[] for i in range(self.FOLDS)]
        self.X_valid_sets=[[] for i in range(self.FOLDS)]
        self.y_valid_sets=[[] for i in range(self.FOLDS)]
        Xt = self.X_train_sets
        yt = self.y_train_sets
        Xv = self.X_valid_sets
        yv = self.y_valid_sets
        with open (FULLPATH,'r') as infile:
            myreader = csv.DictReader(infile)
            for row in myreader:
                fold =  int(row['Fold'])
                split = row['Split']
                case =  row['Case']
                label = int(row['Class'])
                if split == 'Train':
                    Xt[fold].append(case)
                    yt[fold].append(label)
                else:
                    Xv[fold].append(case)
                    yv[fold].append(label)
    def get_train_split(self,fold):
        Xt = self.X_train_sets
        yt = self.y_train_sets
        return Xt[fold],yt[fold]
    def get_valid_split(self,fold):
        Xv = self.X_valid_sets
        yv = self.y_valid_sets
        return Xv[fold],yv[fold]        
    def validate(self):
        print('Validation')
        grow_list = []
        all_vals = []
        partition_size = None
        for i in range(self.FOLDS):
            Xt,yt = self.get_train_split(i)
            Xv,yv = self.get_valid_split(i)
            print('Fold',i,'train/valid',len(Xt),len(Xv))
            if partition_size is None:
                partition_size = len(Xt)+len(Xv)
            else:
                if len(Xt)+len(Xv) != partition_size:
                    print('Unequal partitions')
            if len(yt)!=len(Xt):
                print('Unequal train lengths')
            if len(yv)!=len(Xv):
                print('Unequal valid lengths')
            grow_list += Xt
            grow_list += Xv
            all_vals += Xv
        unique_vals = np.unique (all_vals)
        if len(all_vals) != len(unique_vals):
            print('There must be a duplicated patient in validation')
            ok = False
        all_cases = np.unique ( grow_list )
        for i in range(self.FOLDS):
            ok = True
            Xt,yt = self.get_train_split(i)
            Xv,yv = self.get_valid_split(i)
            my_cases = np.sort ( Xt+Xv )
            for X in all_cases:
                if X not in my_cases:
                    print('Fold',i,'case',X,'is missing')
                    ok = False
            for X in my_cases:
                if X not in all_cases:
                    print('Fold',i,'case',X,'is extra')
                    ok = False
        return True

In [5]:
total = all_splits()
total.load_csv(INFILE)
if total.validate():
    print('ok')
else:
    print('problem!')

Validation
Fold 0 train/valid 101 26
Fold 1 train/valid 101 26
Fold 2 train/valid 102 25
Fold 3 train/valid 102 25
Fold 4 train/valid 102 25
ok
